In [1]:
import numpy as np
import pandas as pd
from flair.data import Sentence
from flair.models import SequenceTagger
import re
from word_forms.word_forms import get_word_forms
from sklearn.metrics import pairwise_distances
import json
from sentence_transformers import SentenceTransformer

import inflect

inflect_engine = inflect.engine()

# Downloading Models and storing them locally

In [2]:
# model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

In [3]:
# model.save('models/ent_name_sim/', 'ent_name_sim')


In [4]:
# tagger = SequenceTagger.load("flair/pos-english")
# tagger.save('models/pos1')

In [5]:
# ner_large = SequenceTagger.load("flair/ner-english-large")
# ner_large.save('models/ner_large')

In [6]:
# ner_base = SequenceTagger.load("flair/ner-english")
# ner_base.save('models/ner_base')

In [7]:
# tagger = SequenceTagger.load('models/pos1')
# ner_large = SequenceTagger.load('models/ner_large')
# ner_base = SequenceTagger.load('models/ner_base')
# model = SentenceTransformer('models/ent_name_sim/')

# Filter crowd source

In [8]:
# crowd_pd = pd.read_csv('Data/crowd_data/crowd_data.tsv', sep='\t')
# crowd_pd

In [9]:
# crowd_pd_filter = crowd_pd.filter(['HITId','WorkTimeInSeconds', 'WorkerId'],axis=1)
# malicious_hits = crowd_pd_filter.loc[crowd_pd_filter.groupby('HITId')['WorkTimeInSeconds'].idxmin()]
# malicious_ids = malicious_hits['WorkerId'].unique()
# malicious_ids

In [10]:
# clean_crowd_pd = crowd_pd.drop(crowd_pd[(crowd_pd['WorkerId'].isin(malicious_ids))].index)
# clean_crowd_pd

In [11]:
# clean_crowd_pd.to_csv("Data/crowd_data/clean_crowd_data.csv")

In [12]:
# ans_filter = crowd_pd.filter(['HITId','AnswerLabel'])
# ans_count = ans_filter.groupby('HITId')['AnswerLabel'].value_counts('counts').to_frame()
# rates = ans_count.groupby('HITId')['AnswerLabel'].max()

In [13]:
# # load csv module
# import json

# j = json.dumps(dict(rates))

# # open file for writing, "w" 
# f = open("Data/crowd_data/rates.json","w")

# # write json object to file
# f.write(j)

# # close file
# f.close()

In [14]:
clean_crowd_pd = pd.read_csv('Data/crowd_data/clean_crowd_data.csv')
clean_crowd_pd

,Unnamed: 0,HITId,HITTypeId,Title,Reward,AssignmentId,WorkerId,AssignmentStatus,WorkTimeInSeconds,LifetimeApprovalRate,Input1ID,Input2ID,Input3ID,AnswerID,AnswerLabel,FixPosition,FixValue
0,0,1,7QT,Is this triple correct or incorrect?,$0.50,1,2133ICYWE97,Submitted,60,99%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
1,1,1,7QT,Is this triple correct or incorrect?,$0.50,2,2133U7HKDLO,Submitted,40,40%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,yes,yes
2,2,1,7QT,Is this triple correct or incorrect?,$0.50,3,928UJANWZ12,Submitted,50,98%,wd:Q11621,wdt:P2142,792910554,2.0,INCORRECT,NaN,NaN
3,3,1,7QT,Is this triple correct or incorrect?,$0.50,4,1726JMZQW,Submitted,80,70%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
4,6,2,7QT,Is this triple correct or incorrect?,$0.50,7,2133ICYWE97,Submitted,120,99%,wd:Q603545,wdt:P2142,4300000,1.0,CORRECT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,296,60,9QT,Is this triple correct or incorrect?,$0.50,297,HHCKW1111,Submitted,200,80%,wd:Q21060270,wdt:P27,wd:Q916,1.0,CORRECT,NaN,NaN
220,297,60,9QT,Is this triple correct or incorrect?,$0.50,298,GGUI83657S,Submitted,120,85%,wd:Q21060270,wdt:P27,wd:Q916,1.0,CORRECT,NaN,NaN
221,300,61,9QT,Is this triple correct or incorrect?,$0.50,301,AALKMII97,Submitted,240,98%,wd:Q1288004,wdt:P1412,wd:Q13330,2.0,INCORRECT,Object,Q1860
222,301,61,9QT,Is this triple correct or incorrect?,$0.50,302,HHCKW1111,Submitted,200,80%,wd:Q1288004,wdt:P1412,wd:Q13330,1.0,CORRECT,NaN,NaN


In [15]:
with open("Data/crowd_data/rates.json", "r") as f:
    rates = json.load(f)

# Loading Graph and other files


In [16]:
with open("Data/images.json", "r") as f:
    images = json.load(f)

In [17]:
import pandas as pd
category2URIID = {
            'PER': {'ids':['Q33999', 'Q10800557', 'Q2526255'], 'cat': 'P106'},
            'MISC': {'ids':['Q11424', 'Q20650540', 'Q29168811', 'Q24862', 'Q24865', 'Q24869'], 'cat': 'P31'},

        }
film_properties = set(pd.read_csv('Data/Film Properties.csv')['res'])

noun_mapper = {
        'cast member' : set(['actor', 'actress']),
        'genre': set(['type', 'kind']),
        'publication date': set(['release', 'date', 'airdate', 'publication', 'launch', 'broadcast']),
        'executive producer': set(['showrunner']),
        'screenwriter': set(['scriptwriter', 'screenplay', 'teleplay', 'writer', 'script', 'scenarist', 'story']),
        'director of photography': set(['cinematographer', 'DOP', 'dop']),
        'film editor': set(['editor']),
        'production designer': set(['designer']),
        'cost': set(['budget']),
        'box office': set(['box', 'earning']),
        'nominated for': set(['nomination', 'award', 'finalist', 'shortlist', 'selection']),
        'official website' : set(['website', 'site']),
        'filming location' : set(['flocation']),
        'narrative website' : set(['nlocation']),
        'production company' : set(['company']),
        'country of origin': set(['origin', 'country'])
    

        }


noun_film_properties = set()
for v in noun_mapper.values():
    noun_film_properties.update(v)

In [18]:
film_properties

{'Box Office Mojo film ID (former scheme)',
 'FilmAffinity ID',
 'Filmportal ID',
 'IMDb ID',
 'Kinopoisk film ID',
 'average shot length',
 'award received',
 'based on',
 'box office',
 'cast member',
 'composer',
 'cost',
 'costume designer',
 'country of origin',
 'director',
 'director of photography',
 'distributed by',
 'duration',
 'executive producer',
 'exploitation visa number',
 'film editor',
 'film script',
 'filming location',
 'genre',
 'lyrics by',
 'main subject',
 'media franchise',
 'narrative location',
 'original film format',
 'original language of film or TV show',
 'part of the series',
 'producer',
 'production designer',
 'publication date',
 'screenwriter',
 'set in period',
 'title'}

In [19]:
noun_film_properties

{'DOP',
 'actor',
 'actress',
 'airdate',
 'award',
 'box',
 'broadcast',
 'budget',
 'cinematographer',
 'company',
 'country',
 'date',
 'designer',
 'dop',
 'earning',
 'editor',
 'finalist',
 'flocation',
 'kind',
 'launch',
 'nlocation',
 'nomination',
 'origin',
 'publication',
 'release',
 'scenarist',
 'screenplay',
 'script',
 'scriptwriter',
 'selection',
 'shortlist',
 'showrunner',
 'site',
 'story',
 'teleplay',
 'type',
 'website',
 'writer'}

In [20]:
import rdflib

from pos_extraction import *
from ner_extraction import *





graph = rdflib.Graph().parse('Data/14_graph.nt', format='turtle')

WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')
WD = rdflib.Namespace('http://www.wikidata.org/entity/')



In [21]:
ner_extractor = NER_extractor()
pos_extractor = POS_extractor()

Loading NER models...
2022-12-14 11:30:08,653 loading file models/ner_large
2022-12-14 11:30:43,333 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
2022-12-14 11:30:45,365 loading file models/ner_base
2022-12-14 11:30:47,792 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
Loading Entity name similarity model...
Loading Title embeddings
Loading POS model...
2022-12-14 11:30:51,884 loading file models/pos1
2022-12-14 11:30:53,346 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD


# Create entity name text embeddings

The next cells goal is to create title embeddings to match spell mistakes on entities

In [22]:
# import csv
# with open('Data/ddis-graph-embeddings/entity_ids.del', 'r') as ifile:
#     ent_list = [ent.split('/')[-1] for idx, ent in csv.reader(ifile, delimiter='\t')]

In [23]:
# from tqdm.notebook import tqdm

In [24]:
# ent2name = {}
# tt = tqdm(ent_list)
# for ent in tt:
#     try:
#         q = f'''        
#         prefix wdt: <http://www.wikidata.org/prop/direct/>
#         prefix wd: <http://www.wikidata.org/entity/>

#         SELECT ?lbl 
#         WHERE {{
#           wd:{ent} rdfs:label ?lbl .
#           FILTER(LANG(?lbl) = "en").
#           }}
#           LIMIT 1'''
#         res = str(list(graph.query(q))[0][0])

#         ent2name[ent] = res
#     except:
#         continue
# print(len(ent2name))

In [25]:
# ent2emb = {}
# ttt = tqdm(ent2name.items())
# for k,v in ttt:
#     emb = model.encode(v).tolist()
#     ent2emb[k] = emb


In [26]:
# em_values = np.array(list(ent2emb.values()))
# em_keys = list(ent2emb.keys())

In [27]:
# np.save('Data/title_embeddings.npy', em_values)

In [28]:
# # load json module
# import json

# # create json object from dictionary
# j = json.dumps(em_keys)

# # open file for writing, "w" 
# f = open("Data/titles.json","w")

# # write json object to file
# f.write(j)

# # close file
# f.close()


In [29]:
# # load json module
# import json

# # create json object from dictionary
# j = json.dumps(ent2name)

# # open file for writing, "w" 
# f = open("Data/ent2name.json","w")

# # write json object to file
# f.write(j)

# # close file
# f.close()

In [30]:
# name2ent = {}
# for k,v in ent2name.items():
#     if name2ent.get(v):
#         name2ent[v].append(k)
#     else:
#         name2ent[v] = [k]

In [31]:
# # create json object from dictionary
# j = json.dumps(name2ent)

# # open file for writing, "w" 
# f = open("Data/name2ent.json","w")

# # write json object to file
# f.write(j)

# # close file
# f.close()

# Modular Code parts (some older versions than in Agent's python files)

In [32]:



def _EntityURI_to_ID(URI_LIST, WD='http://www.wikidata.org/entity/'):
    res = []
    for uri in URI_LIST:

        if WD in uri:
            res.append(re.match("{}(.*)".format(WD), uri)[1])

    return res


def _getEntity_URI_ID(graph, ent, WDT, WD, cat2id):
    query = f'''
        prefix wdt: <http://www.wikidata.org/prop/direct/>
        prefix wd: <http://www.wikidata.org/entity/>

        SELECT ?res
        WHERE{{
            ?res rdfs:label "{ent}"@en.
            }}'''
    URI_LIST = [str(x[0]) for x in list(graph.query(query))]

    entities_ids = _EntityURI_to_ID( URI_LIST, WD)

    res = {}
    for e_id in entities_ids:
        for k, v in cat2id.items():
            g = list(graph.objects(WD[e_id], WDT[v['cat']]))
        
            instancesOf = _EntityURI_to_ID(g, WD)
            if instancesOf and set(v['ids']).intersection(instancesOf):
                if res.get(k):
                    res[k].append({'entity':ent, 'id':e_id})
                else:
                    res[k] = [{'entity':ent, 'id':e_id}]


    return res

def getEntities_URIIDs(graph, entities, WDT, WD, cat2id):
    qres = []
    for e in entities:
        uri_res = _getEntity_URI_ID(graph, e, WDT, WD, cat2id)
        if not uri_res and re.search('-', e):
            uri_res = _getEntity_URI_ID(graph, re.sub('-', '–', e), WDT, WD, cat2id)
        elif not uri_res and re.search('–', e):
            uri_res = _getEntity_URI_ID(graph, re.sub('–', '-', e), WDT, WD, cat2id)
        qres.append(uri_res)
    
    entities_uriID = {}
    for q in qres:
        for k, v in q.items():
            if entities_uriID.get(k):
                entities_uriID[k].extend(v)
            else:
                entities_uriID[k] = v
                
    return entities_uriID

def _get_model_res(model, text):
    sentence = Sentence(testsent)
    model.predict(sentence)
    
    Owords = []
    ent_words = []
    idx = []
    ent = []
    for entity in sentence.get_spans('ner'):
        if entity.get_labels('ner')[0].value in ['PER', 'MISC']:
            ent.append(entity.get_labels('ner')[0].value)
            ent_words.append(testsent[entity.start_position:entity.end_position])
            idx.append((entity.start_position,entity.end_position))

        
    if len(idx) == 1:
        Owords.append(text[0:idx[0][0]])
        Owords.append(text[idx[0][1]:])
    else:
        for i, v in enumerate(idx):

            if i == 0:
                Owords.append(text[0:v[0]])
                continue
            Owords.append(text[idx[i-1][1]:v[0]])    
            if i == len(idx)-1:
                Owords.append(text[v[1]:])
        
    return  ent_words, ent, Owords

def get_entities(text):
    if text[-1] == '?' or text[-1] == '.':
        text = text[:-1]
    
    
    ent_words1, ent1, Owords1 = _get_model_res(ner_large, text)
    ent_words2, ent2, Owords2 = _get_model_res(ner_base, text)
    
    print([ent_words1, ent1, Owords1])
    print([ent_words2, ent2, Owords2])
    
    
    if len(ent_words1) > len(ent_words2) and ent_words2:
        word_group = ent_words2
        entities = ent2
        Owords = Owords2
    else:
        word_group = ent_words1
        entities = ent1
        Owords = Owords1


    print()
    print('NER')
    print(Owords)
    print(word_group)
    print(entities)
    print()

    return  word_group, Owords if entities else [text]


In [33]:
def get_pos(text):

    sentence = Sentence(text)

    tagger.predict(sentence)

    pos_words = []
    pos_tags = []
    for entity in sentence:
        pos_words.append(entity.text)
        pos_tags.append(entity.get_labels('pos')[0].value)

    print('POS')
    print(pos_words)
    print(pos_tags)
    print()
    pop = list(zip(pos_words, pos_tags))

    return pop

In [34]:
# get_pos(testsent)

In [35]:
import re



def _pos_to_word_index(pos_tags, Osent):
    res = {}
    for i in pos_tags:
        w = i[0]
        p = i[1]
        try:
            if re.search(w.lower(), Osent.lower()):
                if res.get(p):
                    res[p].append(w)
                else:
                    res[p] = [w]
        except:
            continue
    return res

def get_relations(pos_tags, Owords, graph, WDT, film_properties):

    properties = set()
    properties.update(noun_film_properties)
    properties.update(film_properties)
    
    Osent = ' '.join(Owords)

    pos_text_dict = _pos_to_word_index(pos_tags, Osent)
    print(pos_text_dict)
    
    res = []
    
    fil = [
        re.search('where', Osent.lower()) and re.search('film', Osent.lower()),
        re.search('location', Osent.lower()) and re.search('film', Osent.lower()),
        re.search('place', Osent.lower()) and  re.search('film', Osent.lower()),
        re.search('shooting', Osent.lower()) and re.search('location', Osent.lower()),
        re.search('shot in', Osent.lower()),
        re.search('filmed in', Osent.lower())
    ]
    
    if any(fil):
        res.append({'relation':'filming location', 'ids': ['P915']})
        
    narr = [
        re.search('where', Osent.lower()) and re.search('narrat', Osent.lower()),
        re.search('where', Osent.lower()) and re.search('set', Osent.lower()),
        re.search('where', Osent.lower()) and re.search('takes place', Osent.lower()),
        re.search('place', Osent.lower()) and re.search('set', Osent.lower()),
        re.search('location', Osent.lower()) and re.search('set', Osent.lower()),
        re.search('location', Osent.lower()) and re.search('narrat', Osent.lower()),
        re.search('set', Osent.lower()) and re.search('work', Osent.lower()),
        
    ]
    if any(narr):
        res.append({'relation':'narrative location', 'ids': ['P840']})
    
    if re.search('MPA', Osent):
        res.append({'relation':'MPAA rating', 'ids': ['P1657']})
        
    if re.search('FSK', Osent):
        res.append({'relation': 'FSK film rating', 'ids': ['P1981']})

        
    picture = [re.search('look like', Osent.lower()),
               re.search('looks like', Osent.lower()),
               re.search('picture', Osent.lower()),
               re.search('poster', Osent.lower())]
        
    if any(picture):
        res.append({'relation': 'IMDb ID', 'ids': ['P345']})
        
    recom = [re.search('recommend', Osent.lower()),
               re.search('recommendation', Osent.lower()),
               re.search('suggest', Osent.lower()),
               re.search('suggestion', Osent.lower())]
        
    if any(recom):
        res.append({'relation': 'recommendation', 'ids': []})

    nouns = []
    for pos in pos_text_dict.keys():

        if pos[:2] == 'VB':
            for w in pos_text_dict[pos]:
                noun_conversions = get_word_forms(w)['n']
                matching_conversions = set(properties).intersection(noun_conversions)
                for noun in matching_conversions:
                    if noun in film_properties:
                        nouns.append(noun)
                    elif noun in noun_film_properties:
                        for k, v in noun_mapper.items():
                            if noun in v:
                                nouns.append(k)
                                break

        elif pos == 'NNS':
            for w in pos_text_dict[pos]:
                noun = inflect_engine.singular_noun(w)
                if noun in film_properties:
                    nouns.append(noun)
                elif noun in noun_film_properties:
                    for k, v in noun_mapper.items():
                        if noun in v:
                            nouns.append(k)
                            break
                
    print(pos_text_dict.get('NN', []))
    for noun in pos_text_dict.get('NN', []):
        if noun in film_properties:
            nouns.append(noun)
        elif noun in noun_film_properties:
            for k, v in noun_mapper.items():
                if noun in v:
                    nouns.append(k)
                    break

    for noun in set(nouns):
        res.append({'relation': noun, 'ids': pos_extractor._getRelation_URI_ID(graph, noun, WDT)})

    return res

In [36]:
# q = f'''prefix wdt: <http://www.wikidata.org/prop/direct/>
# prefix wd: <http://www.wikidata.org/entity/>

# SELECT ?res
# WHERE{{
#   ?res rdfs:label "costume designer"@en.
# }}'''
# list(graph.query(q))

In [37]:
# pos_extractor._getRelation_URI_ID(graph, 'budget', WDT)

In [38]:
# _Relation_URI_ID(uris, WDT)

In [39]:
get_word_forms('Director'.lower())

{'n': {'directive',
  'directiveness',
  'directivenesses',
  'directives',
  'directivities',
  'directivity',
  'directness',
  'directnesses',
  'director',
  'directors',
  'directorship',
  'directorships'},
 'a': {'direct', 'directive'},
 'v': {'direct', 'directed', 'directing', 'directs'},
 'r': {'direct', 'directly'}}

In [40]:
def embeddings(entity_emb, ent2id, relation_emb, rel2id, ent, rel, num_ret):
    professions = set(graph.query(f'''
    prefix wdt: <http://www.wikidata.org/prop/direct/>
    prefix wd: <http://www.wikidata.org/entity/>
    
    SELECT ?obj ?lbl WHERE {{
        wd:{ent} wdt:{rel} ?obj .
        ?obj rdfs:label ?lbl .
    }}
    '''))
    try:
        
        # "Jean Van Hamme" entity
        head = entity_emb[ent2id[WD[ent]]]
        # "occupation" relation
        pred = relation_emb[rel2id[WDT[rel]]]
        # add vectors according to TransE scoring function.
        lhs = head + pred
        # compute distance to *any* entity
        dist = pairwise_distances(lhs.reshape(1, -1), entity_emb).reshape(-1)
        # find most plausible entities
        most_likely = dist.argsort()


        return [{'label':ent2lbl[id2ent[idx]], 'Score': dist[idx]} for idx in most_likely[:num_ret]]
    except:
        return []
    
#     return pd.DataFrame([(id2ent[idx][len(WD):], ent2lbl[id2ent[idx]], dist[idx], rank+1)
#                          for rank, idx in enumerate(most_likely[:10])],
#                         columns=('Entity', 'Label', 'Score', 'Rank'))

In [41]:
# list(graph.objects(WD['Q193835'], WDT['P1411']))
# plots_embeddings
def movie_recom_movie(ent, WD, entity_emb, ent2id, id2ent, ent2lbl):
    m_ids = [p['id'] for p in ent['MISC']]
    
    mean_emb = np.mean([entity_emb[ent2id[WD[idd]]] for idd in m_ids], 0)

    dist = pairwise_distances(mean_emb.reshape(1, -1), entity_emb).reshape(-1)
    # find most plausible entities
    most_likely = dist.argsort()
    
    res = []
    for idx in most_likely[:15+len(m_ids)]:
        g = list(filter(lambda x: str(x).split('/')[-1] in category2URIID['MISC']['ids'], 
                        list(graph.objects(id2ent[idx], WDT.P31))))

        if g and id2ent[idx][len(WD):] not in m_ids and ent2lbl.get(id2ent[idx]):
            res.append({'ent':id2ent[idx][len(WD):], 'label':ent2lbl[id2ent[idx]], 'Score': dist[idx]})
            
    return res[:3]

In [42]:
def movie_recom_genre(graph, genre):
    query = f'''
    prefix wdt: <http://www.wikidata.org/prop/direct/>
    prefix wd: <http://www.wikidata.org/entity/>

    SELECT ?obj ?lbl 
    WHERE {{
      ?obj wdt:P136 wd:{genre} .
      ?obj rdfs:label ?lbl .
      FILTER(LANG(?lbl) = "en").
      }}
    ORDER BY RAND() LIMIT 1
    
    '''
    qres = list(graph.query(query))[0]

    res = {'ent':str(qres[0]).split('/')[-1], 'label': str(qres[1])}

    return res
# movie_recom_genre(graph, 'Q130232')

In [43]:
def movie_recom_actor_genre(graph, actor, genre):
    query = f'''
    prefix wdt: <http://www.wikidata.org/prop/direct/>
    prefix wd: <http://www.wikidata.org/entity/>

    SELECT ?obj ?lbl 
    WHERE {{
      ?obj wdt:P136 wd:{genre}  .
      ?obj wdt:P161 wd:{actor} .
      ?obj rdfs:label ?lbl .
      FILTER(LANG(?lbl) = "en").
    }}
    ORDER BY RAND() LIMIT 1
    
    '''
    qres = list(graph.query(query))[0]

    res = {'ent':str(qres[0]).split('/')[-1], 'label': str(qres[1])}

    return res
movie_recom_actor_genre(graph, 'Q169963', 'Q188473')

{'ent': 'Q1576873', 'label': 'Fast & Furious'}

In [44]:
def movie_recom_actor(graph, actor):
    query = f'''
    prefix wdt: <http://www.wikidata.org/prop/direct/>
    prefix wd: <http://www.wikidata.org/entity/>

    SELECT ?obj ?lbl 
    WHERE {{
      ?obj wdt:P161 wd:{actor} .
      ?obj rdfs:label ?lbl .
      FILTER(LANG(?lbl) = "en").
    }}
    ORDER BY RAND() LIMIT 1
    
    '''
    qres = list(graph.query(query))[0]

    res = {'ent':str(qres[0]).split('/')[-1], 'label': str(qres[1])}

    return res


In [45]:
# # recommendation
# final_ans = ''
# if 'recommendation' in [r['relation'] for r in rel]:
#     if ent.get('MISC'):
#         res = movie_recom_movie(ent, WD, entity_emb, ent2id, id2ent, ent2lbl)
#         ent_print_list = []
#         for r in res:
#             ent_print_list.append(f"{r['label']}{get_movie_date(graph, r['ent'])}")
#         final_ans += f" Hmm... I could recommend you {', '.join(ent_print_list)}."
#     else: 
#         Osent = ' '.join(Owords)
#         genre = ''
#         for v in genre_dict.values():
#             words = v['words']
#             found = False
#             for w in words:
#                 if re.search(w, Osent.lower()):
#                     genre = v['id']
#                     found = True
#                     break
#             if found:
#                 break
                    
#         if ent.get('PER') and genre:
#             for actor in ent['PER']:
#                 res = movie_recom_actor_genre(graph, actor['id'], genre)
#                 ent_print = f"{res['label']}{get_movie_date(graph, res['ent'])}"
#                 final_ans += f" Hmm... I could recommend you {ent_print}."
#         elif ent.get('PER'):
#             for actor in ent['PER']:
#                 res = movie_recom_actor(graph, actor['id'])
#                 ent_print = f"{res['label']}{get_movie_date(graph, res['ent'])}"
#                 final_ans += f" Hmm... I could recommend you {ent_print}."
#         elif genre:
#             res = movie_recom_genre(graph, genre)
#             ent_print = f"{res['label']}{get_movie_date(graph, res['ent'])}"
#             final_ans += f" Hmm... I could recommend you {ent_print}."
    
    
# final_ans

In [46]:
def get_movie_date(graph, ent):
    query = f'''
        prefix wdt: <http://www.wikidata.org/prop/direct/>
        prefix wd: <http://www.wikidata.org/entity/>

        SELECT ?obj 
        WHERE {{
          wd:{ent} wdt:P577  ?obj.
        }} ORDER BY ASC(?obj)
         LIMIT 1
    
    '''
    return f" ({str(max(list(graph.query(query))[0][0].split('-')))})" if list(graph.query(query)) else ''
get_movie_date(graph,'Q1410031')

' (2010)'

In [47]:
def crowdsource_search(ent, rel):
    res = clean_crowd_pd[(clean_crowd_pd['Input1ID'] == f"wd:{ent}") & (clean_crowd_pd['Input2ID'] == f"wdt:{rel}")]

    if bool(list(res.HITId)):
        agg_dict = {}
        for k,v in list(zip(res.Input3ID, res.AnswerLabel)):
            if agg_dict.get(k):
                agg_dict[k].append(v)
            else:
                agg_dict[k] = [v]
                
        agg_res = {}
        for k, v in agg_dict.items():
            agg_res[k] = max(set(v), key=v.count)

        hitid = list(set(res.HITId))[0]
        rate_ans = rates[str(hitid)]
        ans =  str(list(agg_res.keys())[0]).strip('wd:')
        state = list(agg_res.values())[0]
        
        filter_fix_val = set(filter(lambda x: isinstance(x, str), list(res.FixValue)))
        
        if list(agg_res.values())[0] == 'INCORRECT' and filter_fix_val:
            ans = str(list(filter_fix_val)[0]).strip('wd:')
            state = 'CORRECT'
                       
        return ans, rate_ans, state
    else:
        return None, None, None

# crowdsource_search('Q171300','P2142')

In [48]:
# # retrieval
# retrieval = {}
# final_ans = ''

# for k, v in ent.items():

#     for ee in v:
#         retrieval[ee['entity']] = {}
#         for relation in rel:
#             retrieval[ee['entity']][relation['relation']] = []
#             for rid in relation['ids']:
                
#                 g = list(graph.objects(WD[ee['id']], WDT[rid]))
                
#                 if relation['relation'] == 'IMDb ID':
#                     imbds = []
#                     for i in g:
#                         imbds.append(str(i))
#                     imdb_id = imbds[0]

#                     im_id = ''
#                     for image in images:
#                         if k == 'MISC':
#                             if image['movie'] == [imdb_id] and image['type'] == 'poster':
#                                 im_id = image['img']
#                                 im_id = 'image:'+im_id.strip('.jpg')
#                                 break
#                         else:
#                             if image['cast'] == [imdb_id] and image['type'] != 'poster':
#                                 im_id = image['img']
#                                 im_id = 'image:'+im_id.strip('.jpg')
#                                 break


#                     if im_id:
#                         final_ans += f"There you go... {im_id}"
                    
#                 # knowledge graph on these particular relations    
#                 elif relation['relation'] in ['publication date', 'cost', 'box office']:
#                     kg_res = str(g[0]) if g else None
                    
#                     cs_ans, cs_rate, cs_state = crowdsource_search(ee['id'], rid)
                    
#                     if kg_res and cs_ans:
#                         if kg_res == cs_ans:
#                             if cs_state == 'CORRECT' and cs_rate > 0.5:
#                                 final_ans += f" The {relation['relation']} of {ee['entity']} is {kg_res} (Crowd Approval rate: {cs_rate})."
#                             else:
#                                 final_ans += f" I think the {relation['relation']} of {ee['entity']} is {kg_res} but im not really sure (Crowd Approval rate: {cs_rate})."
#                         else:
#                             if cs_state == 'CORRECT' and cs_rate > 0.5:
#                                 final_ans += f" The {relation['relation']} of {ee['entity']} is {cs_ans} (Crowd Approval rate: {cs_rate})."
#                             else:
#                                 final_ans += f" I think the {relation['relation']} of {ee['entity']} is {kg_res} but im not really sure (Crowd Approval rate: {cs_rate})."
#                     elif cs_ans:
#                         if cs_state == 'CORRECT' and cs_rate > 0.5:
#                             final_ans += f" The {relation['relation']} of {ee['entity']} is {cs_ans} (Crowd Approval rate: {cs_rate})."
#                         else:
#                             final_ans += f" I don't really know the {relation['relation']} of {ee['entity']}. But {cs_ans} is not (Crowd Approval rate: {cs_rate})."
#                     elif kg_res:
#                         final_ans += f" The {relation['relation']} of {ee['entity']} is {kg_res}."
                                
                    
#                 else: #knowledge graph
#                     ent_print = f"{ee['entity']}{get_movie_date(graph, ee['id'])}"
#                     print(g)
#                     result = _EntityURI_to_ID(g)
                    
#                     # crowd source for 1 result or no result on KG
#                     cs_ans, cs_rate, cs_state = None, None, None
#                     if len(result) < 2:
#                         cs_ans, cs_rate, cs_state = crowdsource_search(ee['id'], rid)
                        
                        
#                     kg_res = []
#                     for r in result:

#                         query = f'''
#                                 prefix wdt: <http://www.wikidata.org/prop/direct/>
#                                 prefix wd: <http://www.wikidata.org/entity/>

#                                 SELECT ?res
#                                 WHERE
#                                 {{
#                                 wd:{r} rdfs:label ?res .
#                                 FILTER(LANG(?res) = "en").
#                                 }}'''
#                         kg_res.extend([str(i[0]) for i in set(graph.query(query))])
                    
#                     if kg_res and cs_ans:
#                         if kg_res == cs_ans:
#                             if cs_state == 'CORRECT' and cs_rate > 0.5:
#                                 final_ans += f" The {relation['relation']} of {ent_print} is {kg_res} (Crowd Approval rate: {cs_rate})."
#                             else:
#                                 final_ans += f" I think the {relation['relation']} of {ent_print} is {kg_res} but im not really sure (Crowd Approval rate: {cs_rate})."
#                         else:
#                             if cs_state == 'CORRECT' and cs_rate > 0.5:
#                                 final_ans += f" The {relation['relation']} of {ent_print} is {cs_ans} (Crowd Approval rate: {cs_rate})."
#                             else:
#                                 final_ans += f" I think the {relation['relation']} of {ent_print} is {kg_res} but im not really sure (Crowd Approval rate: {cs_rate})."
#                     elif cs_ans:
#                         if cs_state == 'CORRECT' and cs_rate > 0.5:
#                             final_ans += f" The {relation['relation']} of {ent_print} is {cs_ans} (Crowd Approval rate: {cs_rate})."
#                         else:
#                             final_ans += f" I don't really know the {relation['relation']} of {ent_print}. But {cs_ans} is not (Crowd Approval rate: {cs_rate})."
#                     elif kg_res:
#                         if len(kg_res) > 1:
#                             final_ans += f" The {inflect_engine.plural_noun(relation['relation'])} of {ent_print} are {', '.join(kg_res)}."
#                         else:
#                             final_ans += f" The {relation['relation']} of {ent_print} is {kg_res[0]}."
                        
                        
                        
#                     #embedding
#                     n_to_retr = len(kg_res)

#                     emb_res = embeddings(entity_emb, ent2id, relation_emb, rel2id, ee['id'], rid, n_to_retr+1)

#                     if emb_res:
#                         if len(emb_res) > 1:
#                             labels = ', '.join([e['label'] for e in emb_res])
#                             scores = ', '.join([str(e['Score']) for e in emb_res])
#                             final_ans += f" The Embeddings suggest that the {inflect_engine.plural_noun(relation['relation'])} of {ent_print} could be {labels} (Scores: {scores})."
#                         else:
#                             final_ans += f" The Embeddings suggest that the {relation['relation']} of {ent_print} coulb be {emb_res[0]['label']} (Score: {emb_res[0]['Score']})."

                    
# final_ans

In [49]:
# #knowledge graph
# kg = {}
# for e in ent.values():
#     for ee in e:
#         kg[ee['entity']] = {}
#         for relation in rel:
#             kg[ee['entity']][relation['relation']] = []
#             for rid in relation['ids']:
#                 g = list(graph.objects(WD[ee['id']], WDT[rid]))


#                 result = _EntityURI_to_ID(g, WD)
#                 for r in result:
#                     query = f'''
#                             prefix wdt: <http://www.wikidata.org/prop/direct/>
#                             prefix wd: <http://www.wikidata.org/entity/>

#                             SELECT ?res
#                             WHERE
#                             {{
#                             wd:{r} rdfs:label ?res .
#                             FILTER(LANG(?res) = "en").
#                             }}'''

#                     kg[ee['entity']][relation['relation']].extend([str(i[0]) for i in set(graph.query(query))])
# kg

In [50]:
# # embeddings
# import re
# embd = {}
# for e in ent.values():
#     for ee in e:
#         embd[ee['entity']] = {}
#         for relation in rel:
#             embd[ee['entity']][relation['relation']] = []
#             for rid in relation['ids']:
                
#                 n_to_retr = len(kg[ee['entity']][relation['relation']])
                
#                 result = embeddings(entity_emb, ent2id, relation_emb, rel2id, ee['id'], rid, n_to_retr+1)


#                 embd[ee['entity']][relation['relation']].extend(result)
# embd

# Dictionary loading

In [51]:
import rdflib
import numpy as np
import csv
RDFS = rdflib.namespace.RDFS
WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')
WD = rdflib.Namespace('http://www.wikidata.org/entity/')

In [52]:
entity_emb = np.load('Data/ddis-graph-embeddings/entity_embeds.npy')
relation_emb = np.load('Data/ddis-graph-embeddings/relation_embeds.npy')

# load the dictionaries
with open('Data/ddis-graph-embeddings/entity_ids.del', 'r') as ifile:
    ent2id = {rdflib.term.URIRef(ent): int(idx) for idx, ent in csv.reader(ifile, delimiter='\t')}
    id2ent = {v: k for k, v in ent2id.items()}
with open('Data/ddis-graph-embeddings/relation_ids.del', 'r') as ifile:
    rel2id = {rdflib.term.URIRef(rel): int(idx) for idx, rel in csv.reader(ifile, delimiter='\t')}
    id2rel = {v: k for k, v in rel2id.items()}

ent2lbl = {ent: str(lbl) for ent, lbl in graph.subject_objects(RDFS.label)}
lbl2ent = {lbl: ent for ent, lbl in ent2lbl.items()}

In [53]:
genre_dict = {
    'drama film': {'words': ['drama'], 'id': 'Q130232'},
    'documentary film': {'words': ['documentary', 'factual'], 'id': 'Q93204'},
    'comedy film': {'words': ['funny', 'comedy', 'comedic'], 'id': 'Q157443'},
    'crime film': {'words': ['crime'], 'id': 'Q959790'},
    'action film': {'words': ['action'], 'id': 'Q188473'},
    'romance film': {'words': ['romantic', 'romance'], 'id': 'Q1054574'},
    'horror film': {'words': ['horror', 'scary'], 'id': 'Q200092'},
    'adventure film': {'words': ['adventure'], 'id': 'Q319221'},
    'neo-noir': {'words': ['neo-noir', 'new-black', 'neo noir', 'new black'], 'id': 'Q2421031'},
    'science fiction': {'words': ['science fiction', 'SF', 'scifi', 'sci Fi', 'fantasy' 
                                  'sci-Fi', 'science-fiction', 'sci fi', 'sciencefiction'], 'id': 'Q24925'},
    'thriller film': {'words': ['thriller', 'suspense'], 'id': 'Q2484376'},
    'animated film': {'words': ['animated', 'animation', 'cartoon'], 'id': 'Q202866'},


}

In [54]:
from intent_decider import *
intent_dec = IntentionDecider()

Loading Clean crowd data and rates...


# Run AGENT

In [55]:
testsent = "who is the director Good Will Hunting."

testsent = re.sub('executive producer', 'showrunner', testsent)
testsent = re.sub('production designer', 'designer', testsent)
testsent = re.sub('costume designer', 'costume', testsent)
testsent = re.sub('box office', 'box', testsent)
testsent = re.sub('narrative location', 'nlocation', testsent)
testsent = re.sub('filming location', 'flocation', testsent)
testsent = re.sub('production company', 'company', testsent)


# testsent = re.sub(',', ' , ', testsent)
print(testsent)

entities,  Owords = ner_extractor.get_entities(testsent)

pos = pos_extractor.get_pos(testsent)


ent = ner_extractor.getEntities_URIIDs(graph, entities, WDT, WD, category2URIID)
rel = pos_extractor.get_relations(pos, Owords, graph, WDT, film_properties)
print()
print(rel)
print(ent)

who is the director Good Will Hunting.

NER
['who is the director ', '']
['Good Will Hunting']

POS
['who', 'is', 'the', 'director', 'Good', 'Will', 'Hunting', '.']
['WP', 'VBZ', 'DT', 'NN', 'NNP', 'NNP', 'NNP', '.']

{'WP': ['who'], 'VBZ': ['is'], 'DT': ['the'], 'NN': ['director'], '.': ['.']}

[{'relation': 'director', 'ids': ['P57']}]
{'MISC': [{'entity': 'Good Will Hunting', 'id': 'Q193835'}]}


In [56]:
intent_dec.decider(graph, WD, WDT, ent, rel, Owords, entity_emb, ent2id, ent2lbl, id2ent, 
                   relation_emb, rel2id, images, genre_dict, category2URIID)

'The director of Good Will Hunting (1997) is Gus Van Sant. The Embeddings suggest that the directors of Good Will Hunting (1997) could be Harmony Korine, Ben Affleck (Scores: 2721.4583, 2832.0962).'

In [57]:
p = ("Sorry mate, couldn't get you or an answer, " +
"I just respond to stuff about movies. By the way in case it is my fault, " +
"make sure you spell correctly the entities (Case sensitive aswell) and " +
"what you want because I forgot to learn magic " +
"sorceries for spell correction. Better luck next time!")

In [58]:
p

"Sorry mate, couldn't get you or an answer, I just respond to stuff about movies. By the way in case it is my fault, make sure you spell correctly the entities (Case sensitive aswell) and what you want because I forgot to learn magic sorceries for spell correction. Better luck next time!"